In [ ]:
import requests
import base64
import os
import json

In [ ]:
# pip install google-genai pillow
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64, os

In [ ]:
gpt_api_key = ""
gemini_api_key = ""


In [ ]:
path1 = '../Tell2Design Data/centroid_distance/'
path2 = '../Tell2Design Data/direct_adjacency/'
path3 = '../Tell2Design Data/room_removal/'
path4 = '../Tell2Design Data/common_neighbor/'
path5 = '../Tell2Design Data/topological_ordering/'
path6 = '../Tell2Design Data/gemini_description_latest/'

save_path_4 = '../Tell2Design Data/images/centroid_adjacent_removal_neighbor/'
save_path_4_topo = '../Tell2Design Data/images/centroid_adjacent_removal_neighbor_topology/'
save_path_topo = '../Tell2Design Data/images/topological/'
save_path_desc = '../Tell2Design Data/images/description/'

save_path_4_json = '../Tell2Design Data/images/centroid_adjacent_removal_neighbor_json/'
save_path_desc_json = '../Tell2Design Data/images/description_json/'
save_path_topo_json = '../Tell2Design Data/images/topological_json/'
save_path_4_topo_json = '../Tell2Design Data/images/centroid_adjacent_removal_neighbor_topology_json/'

In [ ]:
files = os.listdir(path1)
if '.DS_Store' in files:
    files.remove('.DS_Store')
len(files)

In [ ]:
def generate_image_gpt(text, path):
    url = "https://api.openai.com/v1/images/generations"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {gpt_api_key}"
    }

    payload = {
        "model": "gpt-image-1",
        "prompt": text,
        "size": "auto"
    }

    response = requests.post(url, headers=headers, json=payload)

    #if response.status_code == 200:
    image_base64 = response.json()["data"][0]["b64_json"]
    image_bytes = base64.b64decode(image_base64)
    with open(path, "wb") as f:
        f.write(image_bytes)
        
    #else:
    #    print("Error:", response.status_code, response.text)


In [ ]:
def generate_image_gemini_imagen(path: str, gemini_api_key: str, prompt: str, out_prefix: str = "imagen"):
    client = genai.Client(api_key="")
    response = client.models.generate_images(
        model='imagen-4.0-generate-001',
        prompt=prompt,
        config=types.GenerateImagesConfig(
            number_of_images= 1,
        )
    )
    for generated_image in response.generated_images:
        img_bytes = base64.b64decode(generated_image.image.image_bytes)
        img = Image.open(BytesIO(img_bytes))
        img.save(path)
        

In [ ]:
def generate_concise_text(data, gpt_api_key):
    prompt = [
    {
        "role": "system",
        "content": (
                "You are an expert at condensing step-by-step spatial reasoning into a compact but complete set of facts for floor plan generation."
                "You are given reasoning traces from multiple queries. Your goal is to remove redundant statements while keeping all unique and essential details."
                "Preserve:"
                " - Exact room names as given."
                " - Exact RGB color values for each room, exactly as written in the reasoning."
                " - Positive relations (e.g., “Room X is adjacent to Room Y”, “Room X is north of Room Y”, “Room X is present”)."
                " - Negative relations (e.g., “Room X is not adjacent to Room Y”, “Room X is absent”)."
                " - Positions, directions, and connectivity information."
                " - Topological ordering if given."
                " - Room counts and relative sizes."
                "Do not add new facts or change the meaning of the original reasoning."
                "Output as a single concise text that contains only factual information needed to recreate the floor plan."
        )
    },
    {
        "role": "user",
        "content": (
            f"Reasoning: {data}\n"
            "Return:\n"
            "summary: ..."
        )
    }
    ]

    payload = {
                    "model": "gpt-3.5-turbo",
                    "messages": prompt,
                    "max_tokens": 2000,
                    "temperature": 0.2
                }
    headers = {
                    "Content-Type": "application/json",
                    "Authorization": f"Bearer {gpt_api_key}"
                }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    output = response.json()['choices'][0]['message']['content']
    return output


In [ ]:
for file_i, file_name in enumerate(files):
    print(file_i)
    if file_i < 911:
        continue
    file1 = path1 + file_name
    file2 = path2 + file_name
    file3 = path3 + file_name
    file4 = path4 + file_name
    file5 = path5 + file_name
    file6 = path6 + file_name
    
    data = []
    
    with open(file1, 'r') as f:
        data1 = json.load(f)
        reason1 = data1['Reason']
        data.extend(reason1)
    
    with open(file2, 'r') as f:
        data2 = json.load(f)
        reason2 = data2['Reason']
        data.extend(reason2)
    with open(file3, 'r') as f:
        data3 = json.load(f)
        reason3 = data3['Reason']
        data.extend(reason3)
    with open(file4, 'r') as f:
        data4 = json.load(f)
        reason4 = data4['Reason']
        data.extend(reason4)
    with open(file5, 'r') as f:
        data5 = json.load(f)
        reason5 = data5['Reason']
        data.extend(reason5)
    '''
    with open(file6, 'r') as f:
        data6 = json.load(f)
        reason6 = data6['Reason']
        data.extend(reason6)
    '''
    text = ' '.join(data)
    path = save_path_4_topo + file_name.split('.json')[0] + '.png'
    concise_text = generate_concise_text(text, gpt_api_key)
    result = {
        'concise_text':concise_text
    }
    json_path = save_path_4_topo_json + file_name
    with open(json_path, 'w') as f:
        json.dump(result, f, indent = 2)
    concise_text = "Generate a floor map based on the following context: show the entrance in a small, specified color mentioned only in the text line. "  + concise_text
    #generate_image_gemini_imagen(gemini_api_key, concise_text)
    generate_image_gpt(concise_text, path)
    
    

## 